# Optimization Functions

In [ ]:
import numpy as np

def sphere (x) :
    return np.sum (np.square(x), axis=1)

def matyas (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return 0.26*np.sum(np.square(xy), axis=1) - 0.48*x*y

def bulkin (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return 100*np.sqrt(np.abs(\
                             y - 0.01*np.square(x) 
                             ))
    + 0.01*np.abs(x + 10)

def rastrigin (x) :
    A = 10
    return A*x.shape[1] + np.sum(np.square(x) - A*np.cos(2*np.pi*x) , axis=1)

def alpine (x) :
    return -np.prod (np.sqrt(x)*np.sin(x), axis=1)

def schaffer2 (xy) :
    x2 = np.square (xy[:,0])
    y2 = np.square (xy[:,1])
    return 0.5 + (np.square(np.sin(x2 - y2)) - 0.5)/np.square(1 + 0.001*(x2 + y2))

def schaffer4 (xy) :
    x2 = np.square (xy[:,0])
    y2 = np.square (xy[:,1])
    return 0.5 + (np.square(np.cos(np.sin(np.abs(x2 - y2)))) - 0.5)/np.square(1 + 0.001*(x2 + y2))

def schaffer6 (xy) :
    x = xy[:,0]
    y = xy[:,1]
    sumxy = np.square(x) + np.square(y)
    return 0.5 - (np.square(np.sin(np.sqrt(sumxy))) - 0.5)/np.square(1 + 0.001*sumxy)

def griewank (x) :
    return np.sum(np.square(x), axis=1)/4000 - np.prod(np.cos(x/np.sqrt(np.arange(1, x.shape[1]+1, 1))), axis=1) + 1 
    
def rosenbrock (x) :
    return 100*np.sum(np.square(x[:,1:] - np.square(x[:,:-1])), axis=1) + np.sum(np.square(x[:,:-1]-1), axis=1)

def ackley (x) :
    return np.e + 20 - 20*np.exp(-0.2*np.sqrt(0.5*np.sum(np.square(x), axis=1))) - np.exp(0.5*np.cos(2*np.pi*np.sum(x, axis=1)))

def beale (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(1.5 - x + x*y) + np.square(2.25 - x + x*np.square(y))\
    + np.square(2.625 - x + x*np.power(y, 3))

def goldstein (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return (1+np.square(x+y+1)*(19 - 14*x + 3*np.square(x) - 14*y + 6*x*y + 3*np.square(y)))*\
    (30 + np.square(2*x - 3*y)*(18 - 32*x + 12*np.square(x) + 48*y - 36*x*y + 27*np.square(y)))

def booth (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(x + 2*y - 7) + np.square (2*x + y - 5)

def eggholder (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -(\
    (y+47)*np.sin(np.sqrt(np.abs(\
                                x/2 + y + 47 
                                )))\
    + x*np.sin(np.sqrt(np.abs(\
                              x - y - 47
                             )))
    )

def easom (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -(\
    np.cos(x) * np.cos(y) * np.exp(-(\
                                     np.square(x-np.pi) + np.square(y-np.pi)
                                    ))
    )

def mccormick (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.sin(x+y) + np.square(x-y) - 1.5*x + 2.5*y + 1

def styblinski (x) :
    return np.sum(np.power(x,4) - 16*np.square(x) + 5*x, axis=1)/2

def holdertable (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -np.abs(\
    np.sin(x) * np.cos(y) * np.exp(np.abs(\
                                   1 - np.sqrt(np.square(x) + np.square(y))/np.pi
                                  ))
    )

def crossintray (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -0.0001*np.power(1+np.abs(\
    np.sin(x)*np.sin(y)*np.exp(np.abs(\
                                     100 - np.sqrt(np.square(x) + np.square(y))/np.pi  
                                     )+1)                        
    ),0.1)

def threehumpcamel (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return 2*np.square(x) - 1.05*np.power(x,4) + np.power(x,6)/6 + x*y + np.square(y)

def himmelblau (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(np.square(x) + y - 11) + np.square(x + np.square(y) - 7)

def levi (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(np.sin(3*np.pi*x)) + np.square(x-1)*(1+np.square(np.sin(3*np.pi*y))) + np.square(y-1)*(1+np.square(np.sin(2*np.pi*y)))

## PSO-Class

In [ ]:
class PSO_Optimizer () :
    def __init__ (self, obj, llim, rlim, initgen=None, randgen=None, Np=25) :    
        self.obj = obj
        self.llim = llim
        self.rlim = rlim
        
        D = len(llim)
        self.initgen = (lambda:np.random.random_sample((Np, D))) if initgen is None else initgen
        self.randgen = (lambda:np.random.random_sample((Np, D))) if randgen is None else randgen

        
        self.particles = np.array([l + (r-l)*self.initgen().transpose()[ind] \
                              for ind, l, r in zip(range(0,D), llim, rlim)]).transpose()
        self.velocity = self.initgen()
        
    def ipcd (self, alpha=1.2) :
        particle = self.particles
        part = self.particles + self.velocity
        
        leftvio = part < self.llim
        rightvio = part > self.rlim
        leftRight = np.logical_or(part < self.llim, part > self.rlim)
        vio = np.sum (leftRight, axis=1).astype(bool)
        viosum = np.sum(vio)

        if viosum == 0 :
            self.particles = part
            return

        leftvio = leftvio[vio]
        rightvio = rightvio[vio]
        partV = part[vio]
        particleV = particle[vio]

        limvec = np.copy(partV)
        limvec[leftvio] = np.tile (self.llim, (viosum, 1))[leftvio]
        limvec[rightvio] = np.tile (self.rlim, (viosum, 1))[rightvio]

        diff = partV - particleV
        Xnot = np.sqrt (np.sum (np.square(diff), axis=1, keepdims=True))
        kvec = np.min (np.where (diff == 0, 1, (limvec - particleV)/diff), axis=1, keepdims=True) 

        bvec = particleV + np.where (diff == 0, 0, kvec*diff)
        dvec = np.sqrt (np.sum (np.square(partV - bvec), axis=1, keepdims=True))

        Xpp = dvec*(1 + alpha*np.tan(\
                np.random.rand(viosum).reshape(-1,1)*np.arctan((Xnot - dvec)/(alpha * dvec))))
        boundk = (Xnot - Xpp)/Xnot

        part[vio] = particleV + np.where (diff == 0, 0, boundk*diff)
        
        self.particles = part
        self.velocity[leftRight] *= -1
        
    def optimize (self, w=0.7, c1=2, c2=2, alpha=1.2, beta=0.9, iters=1000) :
        pbest = self.particles
        momentum = np.zeros (shape=pbest.shape)
        gbest = min (pbest , key = lambda x : self.obj(x.reshape(1,-1))[0])

        for i in range (0, iters) :
            r1 = self.randgen()
            r2 = self.randgen()

            # print ("Before = \n" + str(self.particles))
            momentum = beta*momentum + (1-beta)*self.velocity
            self.velocity = momentum + c1*r1*(pbest - self.particles) + c2*r2*(gbest - self.particles)

            self.ipcd ()
            # print ("After = \n" + str(self.particles))
            
            less = self.obj(self.particles) < self.obj(pbest)
            pbest[less] = self.particles[less]
            gbest = min (pbest , key = lambda x : self.obj(x.reshape(1,-1))[0])
        
        return gbest, lambda x : (c1*np.sum(r1) + c2.np.sum(r2))*(gbest - x)/(len(r1)*w)

# Defining Various Optimizers

In [ ]:
import chaosGen as cg
import importlib
cg = importlib.reload (cg)

obj = easom
llim = np.repeat(-5, 2)
rlim = np.repeat(5, 2)

Np = 25
D = len(llim)

swarms = {
    "Rand" : lambda:PSO_Optimizer(obj, llim, rlim), # Random init
    "Lor" : lambda:PSO_Optimizer(obj, llim, rlim, cg.Lorenz((Np,D)).chaosPoints), # Lorenz init
    "Log" : lambda:PSO_Optimier(obj, llim, rlim, cg.Logistic((Np,D)).chaosPoints), # Logistic init
    "Tent" : lambda:PSO_Optimizer(obj, llim, rlim, cg.Tent((Np,D)).chaosPoints), # Tent init
    "RandLog" : lambda:PSO_Optimizer(obj, llim, rlim, None, cg.Logistic((Np,D)).chaosPoints), # Rand init, Log r1r2
    "RandTent" : lambda:PSO_Optimizer(obj, llim, rlim, None, cg.Tent((Np,D)).chaosPoints), # Rand init, Tent r1r2
    "RandLor" : lambda:PSO_Optimizer(obj, llim, rlim, None, cg.Lorenz((Np,D)).chaosPoints), # Rand init, Lorenz r1r2
    "LogLog" : lambda:(lambda cgen:PSO_Optimizer(obj, llim, rlim, cgen, cgen))(cg.Logistic((Np,D)).chaosPoints), # Log init, Log r1r2
    "TentTent" : lambda:(lambda cgen:PSO_Optimizer(obj, llim, rlim, cgen, cgen))(cg.Tent((Np,D)).chaosPoints), # Log init, Log r1r2,
    "LorLor" : lambda:(lambda cgen:PSO_Optimizer(obj, llim, rlim, cgen, cgen))(cg.Lorenz((Np,D)).chaosPoints), # Log init, Log r1r2
}

# swarmOuts = {mizer : lambda:swarms[mizer]().optimize() for mizer in swarms}
# Why doesn't this work?


swarmOuts = {
    "Rand" : lambda:swarms["Rand"]().optimize(),
    "Lor" : lambda:swarms["Lor"]().optimize(),
    "Log" : lambda:swarms["Log"]().optimize(),
    "Tent" : lambda:swarms["Tent"]().optimize(),
    "RandLog" : lambda:swarms["RandLog"]().optimize(),
    "RandTent" : lambda:swarms["RandTent"]().optimize(),
    "RandLor" : lambda:swarms["RandLor"]().optimize(),
    "LogLog" : lambda:swarms["LogLog"]().optimize(),
    "TentTent" : lambda:swarms["TentTent"]().optimize(),
    "LorLor" : lambda:swarms["LorLor"]().optimize(),
}

# Checking Results

In [ ]:
opt = swarmOuts["Rand"]()[0]
print ("f" + str(opt) + " = " + str(obj(opt.reshape(1,-1))[0]))